### In dit script gaan we het BEKK(1,1) model schatten

Om de cel hier onder te runnen moet er een package geinstalleerd worden via anaconda prompt. Deze package zorgt ervoor dat we functies uit andere scripts kunnen gerbuiken in dit script.
<break>
Tik het volgende in anaconda prompt:
<br \>
**pip install ipynb**


In [1]:
%%capture
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from ipynb.fs.full.Dataset import getdata
from ipynb.fs.full.Dataset import getreturns
from scipy.optimize import minimize
from scipy.special import gammaln
import warnings
warnings.filterwarnings('ignore')

###### Hier onder wordt de dataset verkregen. Ik gebruik 2010 tot en met 2015 als mijn in-sample data.
<break>
Hiervoor gebruik een functie die in een ander script (Dataset) is geschreven

In [2]:
df = getreturns()
mInSampleReturns = df.loc[:"2015"]

In de cel hieronder definieren we de update equation van de (diagonal) BEKK(1,1) model en de bijbehorende LogLikelihood functie met normaalverdeling.
Bij de update equation hebben we ook gebruik gemaakt van variance targeting.
Hierdoor ziet de update equation er als volgt uit: 
$$H_t = (ii'-aa'-bb')\circ \bar{H} + A'y_{t-1}y_{t-1}'A + B'H_{t-1}B$$

- Waarbij $a$ en $b$ is gedifinieerd als:
$$a=\begin{pmatrix} a_{11} \\ a_{22} \end{pmatrix}, \quad b=\begin{pmatrix} b_{11} \\ b_{22} \end{pmatrix}$$

- $A$ en $B$ als:
$$A=\begin{pmatrix} a_{11} & 0 \\ 0 & a_{22} \end{pmatrix}, \quad B=\begin{pmatrix} b_{11} & 0 \\ 0 & b_{22} \end{pmatrix} $$

- $\bar{H}$ en $y_t $ als:
$$\bar{H}=\frac{1}{T}\sum_{t=1}^{T}y_ty_t', \quad y_t=\begin{pmatrix} y_{1t} \\ y_{2t} \end{pmatrix} $$

- de bijbehorende Log-Likelihood is als volgt gedefinieerd
$$ LL=-\frac{1}{2}\sum_{t=2}^{T}(n\ln(2\pi) + \ln\left|H_t\right| + y_t'H_t^{-1}y_t), \quad\quad n=2$$
<break>
De functie hierboven wordt gemaximaliseerd wrt $a_{11},a_{22}, b_{11}, b_{22}$

In [5]:
def Parameters_BEKK_Normal(mReturns):
    dA_11 = 0.2
    dA_22 = 0.2
    dB_11 = 0.7
    dB_22 = 0.7    
    vTheta = np.array([dA_11, dA_22, dB_11, dB_22])
    
    def Log_likelihood_BEKK_Normal(vTheta,returns):
        iDimension = 2
        iT = returns.shape[1]
        dA11 = vTheta[0]
        dA22 = vTheta[1]
        dB11 = vTheta[2]
        dB22 = vTheta[3]
        vA = np.zeros( (iDimension,1) )
        vB = np.zeros( (iDimension,1) )
        
        vA[0] = dA11
        vA[1] = dA22
        vB[0] = dB11
        vB[1] = dB22
        
        mA = np.diag(vA.reshape(iDimension,) )
        mB = np.diag(vB.reshape(iDimension,) )
        mOnes = np.ones( (iDimension,iDimension) )
        mH_sum = np.zeros( (iDimension,iDimension) )
        
        for t in range(iT):
            vY_t = returns[:,t]
            vY_t = vY_t.reshape((iDimension,1))
            mH_t = vY_t@vY_t.T
            mH_sum += mH_t
        mH_bar = mH_sum / iT    
        
        mH_old = mH_bar
        dSum = 0
        for t in range(1,iT):
            vY_lag = returns[:,t-1]
            vY_lag = vY_lag.reshape((iDimension,1))
            mY_lag = vY_lag@vY_lag.T
            mH = ( mOnes - vA @ vA.T - vB @ vB.T) * mH_bar + mA.T @ mY_lag @ mA + mB.T @ mH_old @ mB
            
            
            vY = returns[:,t]
            vY = vY.reshape((iDimension,1))
            dLogLik = -0.5 * ( iDimension * np.log(2*np.pi) + np.log(det(mH)) + vY.T @ inv(mH) @ vY )
            dSum += np.asscalar(dLogLik)
            mH_old = mH
        return -dSum
    

    
    def Optimizer(returns, initials, function, bnds):
        result = minimize(function, initials, args=(returns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1), (0, 1))
    result=Optimizer(mReturns, vTheta, Log_likelihood_BEKK_Normal, bounds)
    return result.x, -result.fun, result.success

Hier worden de parameter estimates en de bijbehorende log-likelihood score verkregen.
De volgorde van de paramter vector is:
<break>
$[a_{11},a_{22},b_{11},b_{22}]$

In [6]:
( parameter_N, likelihood_N, success_N) = Parameters_BEKK_Normal( np.array(mInSampleReturns).T )
print(parameter_N)
print(likelihood_N)
print(success_N)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3211.8550334863676
            Iterations: 22
            Function evaluations: 150
            Gradient evaluations: 22
[ 0.25561593  0.24421317  0.9491606   0.95353579]
-3211.8550334863676
True


In de cel hieronder definieren we de update equation van de (diagonal) BEKK(1,1) model en de bijbehorende LogLikelihood functie met student-t verdeling.
Bij de update equation hebben we ook gebruik gemaakt van variance targeting.
Hierdoor ziet de update equation er als volgt uit: 
$$H_t = (ii'-aa'-bb')\circ \bar{H} + A'y_{t-1}y_{t-1}'A + B'H_{t-1}B$$

- Waarbij $a$ en $b$ is gedifinieerd als:
$$a=\begin{pmatrix} a_{11} \\ a_{22} \end{pmatrix}, \quad b=\begin{pmatrix} b_{11} \\ b_{22} \end{pmatrix}$$

- $A$ en $B$ als:
$$A=\begin{pmatrix} a_{11} & 0 \\ 0 & a_{22} \end{pmatrix}, \quad B=\begin{pmatrix} b_{11} & 0 \\ 0 & b_{22} \end{pmatrix} $$

- $\bar{H}$ en $y_t $ als:
$$\bar{H}=\frac{1}{T}\sum_{t=1}^{T}y_ty_t', \quad y_t=\begin{pmatrix} y_{1t} \\ y_{2t} \end{pmatrix} $$

- de bijbehorende Log-Likelihood is als volgt gedefinieerd
$$ LL=-\sum_{t=2}^{T}\left(\log\Gamma\left(\frac{\nu+n}{2}\right) -\log\Gamma\left(\frac{\nu}{2}\right)
        -\frac{1}{2}\log(\pi(\nu-2))-\frac{1}{2}\log(|H_t|)-\left(\frac{\nu+n}{2}\right)                                              \log\left(1+\frac{y_t'H_t^{-1}y_t}{\nu-2}\right)\right), \quad n=2$$
<break>
<break>
De functie hierboven wordt gemaximaliseerd wrt $a_{11},a_{22}, b_{11}, b_{22}, \nu$

In [7]:
def Parameters_BEKK_Student(mReturns):
    dA_11 = 0.1
    dA_22 = 0.1
    dB_11 = 0.8
    dB_22 = 0.8  
    dNu = 4.2
    vTheta = np.array([dA_11, dA_22, dB_11, dB_22,dNu])
    
    def Log_likelihood_BEKK_Student(vTheta,returns):
        iDimension = 2
        iT = returns.shape[1]
        dA11 = vTheta[0]
        dA22 = vTheta[1]
        dB11 = vTheta[2]
        dB22 = vTheta[3]
        dNu = vTheta[4]
        vA = np.zeros((iDimension,1))
        vB = np.zeros((iDimension,1))
        
        vA[0] = dA11
        vA[1] = dA22
        vB[0] = dB11
        vB[1] = dB22
        
        mA = np.diag(vA.reshape(iDimension,))
        mB = np.diag(vB.reshape(iDimension,))
        mOnes = np.ones((iDimension,iDimension))
        mH_sum = np.zeros((iDimension,iDimension))
        
        for t in range(iT):
            vY_t = returns[:,t]
            vY_t = vY_t.reshape((iDimension,1))
            mH_t = vY_t@vY_t.T
            mH_sum += mH_t
        mH_bar = mH_sum/iT    
        
        mH_old = mH_bar
        dSum = 0
        for t in range(1,iT):
            vY_lag = returns[:,t-1]
            vY_lag = vY_lag.reshape((iDimension,1))
            mY_lag = vY_lag@vY_lag.T
            mH = ( mOnes - vA @ vA.T - vB @ vB.T) * mH_bar + mA.T @ mY_lag @ mA + mB.T @ mH_old @ mB
            
            
            vY = returns[:,t]
            vY = vY.reshape((iDimension,1))
            dLogLik = gammaln( (dNu+iDimension)/2 ) - gammaln(dNu/2) - (iDimension/2) * np.log( np.pi*(dNu-2) ) \
                    -(1/2) * np.log( det(mH) ) - ( (dNu+iDimension)/2 ) * np.log(1+ ( ( vY.T @ inv(mH) @ vY ) / ( dNu-2 ) ) )
            dSum += np.asscalar(dLogLik)
            mH_old = mH
        return -dSum
    
    def constraint1(vTheta):
        a11=vTheta[0]
        b11=vTheta[2]
        return 1-a11-b11
    
    def constraint2(vTheta):
        a22=vTheta[1]
        b22=vTheta[3]
        return 1-a22-b22
    
    
    def Optimizer(returns, initials, function, bnds):
        result = minimize(function, initials, args=(returns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1), (0, 1), (2.1,50))
    result = Optimizer(mReturns, vTheta, Log_likelihood_BEKK_Student, bounds)
    return result.x, -result.fun, result.success

Hier worden de parameter estimates en de bijbehorende log-likelihood score verkregen.
De volgorde van de paramter vector is:
<break>
$[a_{11},a_{22},b_{11},b_{22},\nu]$

In [8]:
(parameter_S,likelihood_S,success_S)=Parameters_BEKK_Student(np.array(mInSampleReturns).T)
print(parameter_S)
print(likelihood_S)
print(success_S)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3167.0664203858887
            Iterations: 26
            Function evaluations: 204
            Gradient evaluations: 26
[ 0.23980112  0.23423544  0.95731286  0.95977162  8.15825699]
-3167.0664203858887
True
